In [76]:
import pandas as pd
import premorph
from morph import *

In [102]:
par = pd.read_excel('par_cls.xlsx', encoding='utf-8', header=None)
par.columns = ['id', 'role', 'note', 'end']
root_dict = pd.read_excel('root_dict.xlsx', encoding='utf-8', header=None)
root_dict.columns = ['begin', 'end', 'id']

phrase = 'Кохайтеся чорнобриві, та не з москалями. Москалі лихі люди, роблять лихо з вами.'

In [103]:
print(root_dict)

        begin   end  id
0        коха  тися   1
1   чорнобрив    ий   2
2          та   NaN   3
3          не   NaN   4
4           з   NaN   5
5      москал     ь   6
6          бо   NaN   5
7         чуж    ий   2
8         люд     и   7
9         роб   ити   8
10        лих     о   9
11          в     и  10


In [104]:
# Morpho
words = []

for i in premorph.premorph(phrase):
    for word in list(i):
        for beg in root_dict['begin']:
            if word.startswith(beg):
                words.append(Word(
                        word,
                        int(root_dict.loc[root_dict['begin'] == beg]['id']),
                        beg,
                        word[len(beg):] if len(word[len(beg):]) > 0 else ''
                    ))

In [105]:
for word in words:
    for index, row in par.iterrows():
        if row['end'] == word.ending and row['id'] == word.id:
            word.role = row['role']
        elif (type(row['end']) is not str) and row['id'] == word.id and word.ending == '':
            word.role = row['role']

In [100]:
for word in words:
    print(word.word
          , word.role)

кохайтеся ДЛ
чорнобриві ПЙ
та С
не Ч
з П
москалями ІР
москалі ІС
лихі None
люди МВ
роблять СЧ
лихо ЛШ
з П
вами ЗЕ
